# Movies Recommend System

### 構成

1. レビューの形態素解析
2. 特徴量行列の作成
3. 事前に作成したモデルで予測

### 準備

In [24]:
#パンダスの準備
import pandas as pd
from pandas import Series,DataFrame

In [25]:
#SQLの準備
import sqlite3
con = sqlite3.connect("movies.db")

In [26]:
# 関数を作って、SQL文の実行結果をDataFrameにして返します。
def sql_to_df(sql_query):

    # read_sqlの引数に、SQL文とデータベースへのConnectionを渡します。
    df = pd.read_sql(sql_query, con)

    # 結果のDataFrameを返します。
    return df

In [27]:
import MeCab
mecab = MeCab.Tagger ('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

## 1. レビューの形態素解析

In [88]:
watch = '''
コメディ映画。めちゃくちゃ笑いたい。イギリスっぽいアイロニーを含んだ映画がみたい。
知的な笑いで、パティントンとかそういう映画。
テッドとか下品な映画ではなくて、上品な笑いがいい。
'''

In [89]:
text = watch
mecab.parse('')#文字列がGCされるのを防ぐ
node = mecab.parseToNode(text)
i_id=0
while node:
    #単語を取得
    word = node.surface
    #品詞を取得
    pos = node.feature.split(",")[1]
    #print('{}'.format(word))
    #print('{0} , {1}'.format(word, pos))
        
    #次の単語に進める
    node = node.next
    
    sql_query = '''
    INSERT INTO watch_cnt_table(id, word, num) values({i_id}, "{word}", 1)
    '''.format(i_id=i_id, word=word)
        
    con.execute(sql_query)
    
i_id += 1

In [90]:
sql_query = '''
SELECT * FROM watch_cnt_table
'''
words_used_df = sql_to_df(sql_query)

In [91]:
sql_query = '''
SELECT num_words.id, watch_cnt_table.word FROM watch_cnt_table, num_words WHERE num_words.word = watch_cnt_table.word GROUP BY num_words.id, watch_cnt_table.word ORDER BY num_words.id
'''
words_df = sql_to_df(sql_query)

In [92]:
words_df

,id,word
0,1,。
1,2,、
2,3,が
3,6,は
4,7,て
5,10,を
6,12,で
7,17,な
8,21,
9,27,映画


In [93]:
query_all = []

n = 0
     
query = '''SELECT '''
for item in range(len(words_df)):
    #レビューIDごとに出現しているwordでカウントする。
    query += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{n}, '''.format(word=words_df.iloc[item, 1], n=words_df.iloc[item, 0])
    
query += '''id FROM watch_cnt_table GROUP BY id'''
#print(query)
query_all.append(query)

In [94]:
query_all

["SELECT SUM(CASE WHEN word = '。' THEN num ELSE 0 END) AS a1, SUM(CASE WHEN word = '、' THEN num ELSE 0 END) AS a2, SUM(CASE WHEN word = 'が' THEN num ELSE 0 END) AS a3, SUM(CASE WHEN word = 'は' THEN num ELSE 0 END) AS a6, SUM(CASE WHEN word = 'て' THEN num ELSE 0 END) AS a7, SUM(CASE WHEN word = 'を' THEN num ELSE 0 END) AS a10, SUM(CASE WHEN word = 'で' THEN num ELSE 0 END) AS a12, SUM(CASE WHEN word = 'な' THEN num ELSE 0 END) AS a17, SUM(CASE WHEN word = '' THEN num ELSE 0 END) AS a21, SUM(CASE WHEN word = '映画' THEN num ELSE 0 END) AS a27, SUM(CASE WHEN word = 'だ' THEN num ELSE 0 END) AS a28, SUM(CASE WHEN word = 'いい' THEN num ELSE 0 END) AS a80, SUM(CASE WHEN word = 'なく' THEN num ELSE 0 END) AS a107, SUM(CASE WHEN word = 'とか' THEN num ELSE 0 END) AS a165, SUM(CASE WHEN word = 'たい' THEN num ELSE 0 END) AS a192, SUM(CASE WHEN word = 'みたい' THEN num ELSE 0 END) AS a233, SUM(CASE WHEN word = 'そういう' THEN num ELSE 0 END) AS a387, SUM(CASE WHEN word = 'コメディ' THEN num ELSE 0 END) AS a492, SUM(CA

In [97]:
all_df = sql_to_df(query_all[0])

In [98]:
all_df

,a1,a2,a3,a6,a7,a10,a12,a17,a21,a27,...,a492,a754,a976,a1079,a2431,a2648,a3463,a3691,a7065,id
0,5,2,2,1,1,1,2,3,2,4,...,1,3,1,1,1,1,1,1,1,0


## 2. 特徴量行列の作成

CREATE TABLE num_words( id int primary key, word char(30) );

In [69]:
sql_query = '''
SELECT num_words.id, watch_cnt_table.word FROM watch_cnt_table, num_words WHERE num_words.word = watch_cnt_table.word ORDER BY num_words.id
'''
words_df = sql_to_df(sql_query)

In [70]:
words_df

,id,word
0,1,。
1,1,。
2,1,。
3,1,。
4,1,。
5,2,、
6,2,、
7,3,が
8,3,が
9,6,は


同じ順番でワードをとってくるというかんじ

In [50]:
l = []
for num in range(0,30400):
    literal = 'a{num}'.format(num=num)
    l.append(literal)

In [51]:
l

['a0',
 'a1',
 'a2',
 'a3',
 'a4',
 'a5',
 'a6',
 'a7',
 'a8',
 'a9',
 'a10',
 'a11',
 'a12',
 'a13',
 'a14',
 'a15',
 'a16',
 'a17',
 'a18',
 'a19',
 'a20',
 'a21',
 'a22',
 'a23',
 'a24',
 'a25',
 'a26',
 'a27',
 'a28',
 'a29',
 'a30',
 'a31',
 'a32',
 'a33',
 'a34',
 'a35',
 'a36',
 'a37',
 'a38',
 'a39',
 'a40',
 'a41',
 'a42',
 'a43',
 'a44',
 'a45',
 'a46',
 'a47',
 'a48',
 'a49',
 'a50',
 'a51',
 'a52',
 'a53',
 'a54',
 'a55',
 'a56',
 'a57',
 'a58',
 'a59',
 'a60',
 'a61',
 'a62',
 'a63',
 'a64',
 'a65',
 'a66',
 'a67',
 'a68',
 'a69',
 'a70',
 'a71',
 'a72',
 'a73',
 'a74',
 'a75',
 'a76',
 'a77',
 'a78',
 'a79',
 'a80',
 'a81',
 'a82',
 'a83',
 'a84',
 'a85',
 'a86',
 'a87',
 'a88',
 'a89',
 'a90',
 'a91',
 'a92',
 'a93',
 'a94',
 'a95',
 'a96',
 'a97',
 'a98',
 'a99',
 'a100',
 'a101',
 'a102',
 'a103',
 'a104',
 'a105',
 'a106',
 'a107',
 'a108',
 'a109',
 'a110',
 'a111',
 'a112',
 'a113',
 'a114',
 'a115',
 'a116',
 'a117',
 'a118',
 'a119',
 'a120',
 'a121',
 'a122',
 'a

In [71]:
for item in range(len(words_df)):
    print(words_df.iloc[item, 0])

1
1
1
1
1
2
2
3
3
6
7
10
12
12
17
17
17
21
21
27
27
27
27
28
80
107
165
165
192
233
387
492
754
754
754
976
1079
2431
2648
3463
3691
7065


In [53]:
l_data = []
for item in range(len(words_df)):
    for num in range(0,30400):
        if words_df[id] == num:
            l_data.append(words_df.iloc[item, 0])
        else:
            l_data.append(0)

KeyError: <built-in function id>

In [13]:
all_df = pd.DataFrame(l_data, columns = l)

In [14]:
all_df

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a31557,a31558,a31559,a31560,a31561,a31562,a31563,a31564,a31565,a31566


In [137]:
words_used_df.tail(1)

,word
31566,￣*


In [138]:
31566 / 1900

16.613684210526316

In [145]:
all_df = all_df.drop('id', axis=1)

In [146]:
all_df

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a30390,a30391,a30392,a30393,a30394,a30395,a30396,a30397,a30398,a30399
0,0,5,2,2,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
con.close

<function Connection.close>

## 3. 事前に作成したモデルで予測

In [99]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline

In [100]:
from sklearn import linear_model

参考 : http://qiita.com/yshi12/items/9f3f8ae69588da9f018f,  http://techblog.yahoo.co.jp/datascience/use_scikit-learn/

scikit-learnを使った多クラス分類


In [101]:
from sklearn.externals import joblib
#モデルの読み込み
logreg = joblib.load("model_dir/model")

In [102]:
all_numpyMatrix_data = all_df.as_matrix()
X_test = all_numpyMatrix_data
_test_data = DataFrame(X_test)

In [103]:
# テストデータを予測します。
Y_pred = logreg.predict(X_test)

ValueError: X has 27 features per sample; expecting 30400

In [152]:
Y_pred

array(['アダムス・ファミリー'], dtype=object)

In [127]:
watch[1:91]

'コメディ映画。めちゃくちゃ笑いたい。イギリスっぽいアイロニーを含んだ映画がみたい。\n知的な笑いで、パティントンとかそういう映画。\nテッドとか下品な映画ではなくて、上品な笑いがいい。'

In [128]:
pred = Y_pred[0]

In [122]:
pred

'アダムス・ファミリー'

In [129]:
query = '''
INSERT INTO want(watn, pred) values('{watch}', '{pred}')
'''.format(watch=watch[1:91], pred=pred)

In [ ]:
con.execute(query)

In [123]:
sql_query = '''
SELECT * FROM want
'''
want_df = sql_to_df(sql_query)

In [124]:
want_df

,id,watn,pred
0,None,爽やかな映画。若者が活躍する映画,グッド・ウィル・ハンティング／旅立ち
1,None,そこはかとなく暗くどん底の映画。,エド・ウッド


In [23]:
con.close()

In [130]:
query

"\nINSERT INTO want(watn, pred) values('コメディ映画。めちゃくちゃ笑いたい。イギリスっぽいアイロニーを含んだ映画がみたい。\n知的な笑いで、パティントンとかそういう映画。\nテッドとか下品な映画ではなくて、上品な笑いがいい。', 'アダムス・ファミリー')\n"